# LeNet-5

In [1]:
# 파이썬 2와 파이썬 3 지원
from __future__ import division, print_function, unicode_literals

# 공통
import numpy as np
import os

# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# 맷플롯립 설정
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
plt.rcParams['font.family'] = 'NanumBarunGothic'
plt.rcParams['axes.unicode_minus'] = False

# 그림을 저장할 폴더
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)
    
import tensorflow as tf
import numpy as np

/home/shin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 32*32*3) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 32*32*3) / 255.0
y_train = y_train.astype(np.int32).reshape(-1)
y_test = y_test.astype(np.int32).reshape(-1)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]
m,n = X_train.shape
print(y_train.shape)
print(X_train.shape)
X_test = X_test[:5000]
y_test = y_test[:5000]

(45000,)
(45000, 3072)


In [4]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [5]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size)
    X_batch = X_train[indices]
    y_batch = y_train[indices] 
    return X_batch, y_batch

# 배치정규화

위에서의 ELU와 HE 초기화를 사용하면 훈련 초기 단계에서 그래디언트소실이나 폭주 문제를 크게 감소시킬 수 있지만, 훈련하는 동안 다시 발생하지 않을 수 없습니다. 그렇기 때문에 그래디언트 소실 및 폭주 문제를 해결하기 위해서 배치정규화batch Normalization(BN)을 제안합니다. 이는 훈련하는 동안 이전 층의 파라미터가 변함에 따라 각 층에 들어오는 입력의 분포가 변화되는 문제입니다.(내부 공변량 변화internal Covariate Shift)

이 방법은 각 층에서 활성화 함수를 통과하기전에 모델에 연산을 하나 추가합니다. 단순하게 입력데이터의 평균을 0으로 만들고 정규화한 다음, 각 층에서 두 개의 새로운 파라미터로 결과값의 스케일을 조정하고 이동시킵니다.(p.361) 

테스트 할 떄는 평균과 표준편차를 계산할 미니배치가 없으니 전체 훈련 세트의 평균과 표준편차를 대신 사용합니다. 

전체적으로 보면 배치 정규화된 층마다 스케일, 이동, 평균, 표준편차 네 개의 파라미터가 학습됩니다. 그러나 단점이라면 층마다 추가되는 계산이 신경망의 예측이 느려지게 합니다. 


training = tf.placeholder_with_default(False, shape=(), name = "training")

- batch_normalization()을 할 때 처음 훈련시킬 때는 training값을 가져와 사용하지만 이후 실험데이터에서는 training값을 사용하지 않는다. 그래서 사용하는 placeholder가 이것이다. 

tf.layers.batch_normalization(x, training=training, momentum=0.9)

- BN알고리즘은 지수감소를 이용해 이동평균을 계산한다. 그래서 momentum 매개변수가 필요하다.
- 일반적인 모멘텀 값은 1에 가까운 .9, 0.99, 0.999 이다. (데이터가 크고 미니베치가 작을 경우 9를 더 넣어 1에 가깝게 한다.)

In [6]:
import tensorflow as tf
import numpy as np

height = 32
width = 32
channels = 3
n_inputs = height * width * 3

conv1_fmaps = 96
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "VALID"

pool2_fmap = conv1_fmaps

conv3_fmaps = 128
conv3_ksize = 5
conv3_stride = 1
conv3_pad = "VALID"

pool4_fmap = conv3_fmaps

conv5_fmaps = 256
conv5_ksize = 5
conv5_stride = 1
conv5_pad = "VALID"


n_fc1 = 84
n_outputs = 10

#training = tf.placeholder()

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

#BN을 첨가시킴, act = relu
with tf.name_scope("conv1"):
    conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad, name="conv1")
    
    bn1 = tf.layers.batch_normalization(conv1, training=True, momentum=0.9)
    
    bn1_act = tf.nn.selu(bn1)


with tf.name_scope("pool2"):
    pool2 = tf.nn.max_pool(bn1_act, ksize=[1,2,2,1], strides=[1,2,2,1], padding="VALID")

with tf.name_scope("conv3"):
    conv3 = tf.layers.conv2d(pool2, filters=conv3_fmaps, kernel_size=conv3_ksize,
                         strides=conv3_stride, padding=conv3_pad, name="conv3")
    
    bn2 = tf.layers.batch_normalization(conv3, training=True, momentum=0.9)
    
    bn2_act = tf.nn.selu(bn2)

with tf.name_scope("pool4"):
    pool4 = tf.nn.max_pool(bn2_act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    
with tf.name_scope("conv5"):
    conv5 = tf.layers.conv2d(pool4, filters=conv5_fmaps, kernel_size=conv5_ksize,
                            strides=conv5_stride, padding=conv5_pad, name="conv5", )
    
    bn3 = tf.layers.batch_normalization(conv5, training=True, momentum=0.9)
    
    bn3_act = tf.nn.selu(bn3)
    
with tf.name_scope("pool_flat"):
    pool_flat = tf.reshape(bn3_act, shape=[-1, conv5_fmaps*1])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool_flat, n_fc1,  name="fc1")
    
    bn4 = tf.layers.batch_normalization(fc1, training=True, momentum=0.9)
    
    bn4_act = tf.nn.selu(bn4)

with tf.name_scope("output"):
    logits = tf.layers.dense(bn4_act, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [7]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [8]:
mse_summary = tf.summary.scalar('accuracy', loss)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [9]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [10]:
n_epochs = 20
batch_size = 100
n_batches = int(np.ceil(m / batch_size))
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session(config=config) as sess:
    init.run()
    for epoch in range(n_epochs):
#         for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
#             sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run([training_op, extra_update_ops], feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "배치 데이터 정확도:", acc_batch, "검증 세트 정확도:", acc_val)

    acc_test = accuracy.eval(feed_dict={X: X_test,
                                        y: y_test})
    print("테스트 세트에서 최종 정확도:", acc_test)
        
    save_path = saver.save(sess, "./my_mnist_model")
    
    file_writer.close()

0 배치 데이터 정확도: 0.7 검증 세트 정확도: 0.6046
1 배치 데이터 정확도: 0.77 검증 세트 정확도: 0.643
2 배치 데이터 정확도: 0.79 검증 세트 정확도: 0.6712
3 배치 데이터 정확도: 0.81 검증 세트 정확도: 0.6966
4 배치 데이터 정확도: 0.88 검증 세트 정확도: 0.6986
5 배치 데이터 정확도: 0.82 검증 세트 정확도: 0.7002
6 배치 데이터 정확도: 0.9 검증 세트 정확도: 0.7134
7 배치 데이터 정확도: 0.89 검증 세트 정확도: 0.7164
8 배치 데이터 정확도: 0.86 검증 세트 정확도: 0.7172
9 배치 데이터 정확도: 0.95 검증 세트 정확도: 0.7094
10 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.7124
11 배치 데이터 정확도: 0.97 검증 세트 정확도: 0.7106
12 배치 데이터 정확도: 0.99 검증 세트 정확도: 0.71
13 배치 데이터 정확도: 0.97 검증 세트 정확도: 0.7126
14 배치 데이터 정확도: 0.99 검증 세트 정확도: 0.714
15 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.7122
16 배치 데이터 정확도: 0.99 검증 세트 정확도: 0.7174
17 배치 데이터 정확도: 0.95 검증 세트 정확도: 0.7042
18 배치 데이터 정확도: 0.99 검증 세트 정확도: 0.704
19 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.7184
테스트 세트에서 최종 정확도: 0.7188
